<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter

    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 81kB 4.0MB/s 
    100% |████████████████████████████████| 133kB 2.0MB/s 
  Running setup.py bdist_wheel for oauthlib ... - \ done
  Stored in directory: /Users/kiefer/Library/Caches/pip/wheels/84/98/7a/fba7268f61097bea6081cbe5480bc439b38975748ea7684fd5
Successfully built oauthlib


## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [2]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [5]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [6]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [7]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [9]:
print sanders[0]

{'handle': u'Bernie Sanders', 'mined_at': datetime.datetime(2017, 3, 29, 14, 7, 23, 446404), 'created_at': u'Wed Mar 29 15:05:27 +0000 2017', 'tweet_id': 847102410615074818, 'text': u'RT @Newsweek: How Bernie Sanders\u2019 ideas are winning in the Trump era https://t.co/9y7mc82PFH https://t.co/4DkCh4cijs', 'retweet_count': 582}


In [10]:
print donald[0]

{'handle': u'Donald J. Trump', 'mined_at': datetime.datetime(2017, 3, 29, 14, 7, 24, 64203), 'created_at': u'Wed Mar 29 12:21:02 +0000 2017', 'tweet_id': 847061031293779969, 'text': u'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!', 'retweet_count': 13582}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [12]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Mar 29 15:05:27 +0000 2017,Bernie Sanders,2017-03-29 14:07:23.446404,582,RT @Newsweek: How Bernie Sanders’ ideas are wi...,847102410615074818
1,Tue Mar 28 18:44:43 +0000 2017,Bernie Sanders,2017-03-29 14:07:23.446419,18144,Trump’s executive order is a threat to the wel...,846795201624657921
2,Sat Mar 25 16:01:00 +0000 2017,Bernie Sanders,2017-03-29 14:07:23.578100,4705,We now must take on a horrific Republican budg...,845666839455711234
3,Sat Mar 25 15:20:00 +0000 2017,Bernie Sanders,2017-03-29 14:07:23.578109,8013,We must be clear. This important victory again...,845656521178398720
4,Fri Mar 24 22:09:56 +0000 2017,Bernie Sanders,2017-03-29 14:07:23.718598,555,RT @allinwithchris: Joining @chrislhayes tonig...,845397293234884608


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [13]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [14]:
trump_df = pd.DataFrame(trump_tweets)
print trump_df.shape

(999, 6)


In [16]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [17]:
bernie_df = pd.DataFrame(bernie_tweets) 
print bernie_df.shape

(1000, 6)


In [18]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1999, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'https co', 476),
 (u'will be', 77),
 (u'of the', 67),
 (u'thank you', 67),
 (u'in the', 52),
 (u'we will', 40),
 (u'make america', 39),
 (u'great again', 37),
 (u'fake news', 36),
 (u'going to', 36),
 (u'to the', 35),
 (u'for the', 34),
 (u'america great again', 32),
 (u'america great', 32),
 (u'it is', 32),
 (u'make america great again', 31),
 (u'make america great', 31),
 (u'at the', 30),
 (u'we are', 29),
 (u'draintheswamp https co', 27)]

### Look at the ngrams for Bernie Sanders

In [22]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'https co', 412),
 (u'health care', 75),
 (u'of the', 69),
 (u'donald trump', 63),
 (u'in the', 57),
 (u'going to', 49),
 (u'for the', 45),
 (u'we are', 43),
 (u'bernie sanders', 42),
 (u'to the', 42),
 (u'we must', 40),
 (u'we need', 38),
 (u'is the', 37),
 (u'on the', 33),
 (u'is not', 32),
 (u'stand up', 31),
 (u'millions of', 30),
 (u'should be', 29),
 (u'this country', 29),
 (u'and the', 29)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [25]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [26]:
print tweet_text[0:3]

[ u'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!'
 u'Remember when the failing @nytimes apologized to its subscribers, right after the election, because their coverage was so wrong. Now worse!'
 u"Why doesn't Fake News talk about Podesta ties to Russia as covered by @FoxNews or money from Russia to Clinton - sale of  Uranium?"]


In [27]:
print clean_text[0:3]

[u'if the people of our great country could only see how viciously and inaccurately my administration is covered by certain media', u'remember when the failing nytimes apologized to its subscribers right after the election because their coverage was so wrong now worse', u'why doesnt fake news talk about podesta ties to russia as covered by foxnews or money from russia to clinton sale of uranium']


In [39]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values
print np.mean(y)

0.499749874937


In [40]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(1999, 2000)


In [41]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

import numpy as np
print accuracies
print np.mean(accuracies)

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[ 0.865       0.925       0.915       0.92        0.91        0.89        0.8
  0.9         0.92        0.82914573]
0.887414572864


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [42]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)


array([[ 0.84421873,  0.15578127],
       [ 0.21951849,  0.78048151]])

In [43]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [44]:
# We deviate from trump / sanders using student tweets here to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print "Mining tweets for: ", twitter_handle
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for:  dril
Mining tweets for:  LaziestCanine
Mining tweets for:  nycguidovoice
Mining tweets for:  ch000ch


In [47]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))

print multi.shape


(5994, 6)


In [48]:
multi.handle.value_counts()

chuuch      2000
Lazy dog    1999
wint        1995
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [59]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [61]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.0s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [64]:
# Random forest score:
print 'RF:', rf.score(X_test, y_test)
print 'KNN:', knn.score(X_test, y_test)

RF:

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.4s finished


 0.689827682046
KNN: 0.392996108949


In [66]:
# Baseline score:
multi.handle.value_counts()/multi.shape[0]

chuuch      0.333667
Lazy dog    0.333500
wint        0.332833
Name: handle, dtype: float64

In [67]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [68]:
from sklearn.metrics import classification_report, confusion_matrix

print classification_report(y_test, rf_yhat)


             precision    recall  f1-score   support

          0       0.48      0.20      0.28       636
          1       0.36      0.73      0.48       593
          2       0.46      0.26      0.33       570

avg / total       0.43      0.39      0.36      1799



In [69]:
# Confusion Matrix
print confusion_matrix(y_test, rf_yhat)

[[126 416  94]
 [ 83 435  75]
 [ 53 371 146]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [71]:
rf.fit(X, y)
pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    8.0s finished
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.8s finished


In [72]:
pp[0:5]

array([[ 0.92 ,  0.032,  0.048],
       [ 0.872,  0.076,  0.052],
       [ 0.865,  0.028,  0.107],
       [ 0.864,  0.028,  0.108],
       [ 0.668,  0.268,  0.064]])

In [73]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [75]:
print multi.shape, pp.shape

(5994, 6) (5994, 3)


In [76]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Wed Mar 29 20:04:35 +0000 2017,wint,2017-03-29 15:23:38.392783,718,i have squandered the last of my fathers good ...,847177688485937152,0,0.920,0.032,0.048
1,1,Wed Mar 29 11:32:52 +0000 2017,wint,2017-03-29 15:23:38.392794,143,https://t.co/JzTVdxitfx the state of owl sfx ...,847048911864909824,1,0.872,0.076,0.052


In [82]:
print 'Most dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0]
print 'Least dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0]

Most dril: @theVargest shut the fuck up
Least dril: @antonwheel thanks


In [83]:
print 'Most LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0]
print 'Least LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0]

Most LaziestCanine: My body is my journal, and my tattoos are my story https://t.co/81e09oM7km
Least LaziestCanine: @Just_Benj no doubt


In [86]:
print 'Most chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0]
print 'Least chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0]

Most chuuch: @nanglish @Kyle_Lippert :]
Least chuuch: https://t.co/jBwdrg71lA
